# MAY 22, after Alex' findings

The "orthodox" method does not seem to work with our project.
Alex and Noel found a workaround. We are testing it as a preliminary step
to dropping it in LangChain usage.

#### Required:

```
pip install google-api-python-client~=2.86.0 google-cloud-aiplatform~=1.25.0 google-generativeai~=0.1.0rc3
```

### Authentication

You need a service account, then you run this command on your laptop:

```
gcloud auth login
```

A browser will appear to do the oAuth2 thing. This also creates a JSON file in
`$HOME/.config/gcloud/application_default_credentials.json` (besides printing your google Project ID).

Make these two strings available to the environment (check the following).

In [1]:
import os
import json

from google.cloud import aiplatform
from google.oauth2 import service_account
from vertexai.preview.language_models import TextEmbeddingModel, ChatModel, InputOutputTextPair

In [2]:
GOOGLE_CLOUD_PROJECT = os.environ['GOOGLE_CLOUD_PROJECT']
GOOGLE_AUTH_JSON_PATH = os.environ['GOOGLE_AUTH_JSON_PATH']


json_str = open(GOOGLE_AUTH_JSON_PATH).read()
json_data = json.loads(json_str)
json_data['private_key'] = json_data['private_key'].replace('\\n', '\n')

In [3]:
credentials = service_account.Credentials.from_service_account_info(json_data)

aiplatform.init(project=GOOGLE_CLOUD_PROJECT, credentials=credentials)

### Completion model

See for instance `https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.preview.language_models.TextGenerationModel`

In [4]:
from vertexai.preview.language_models import TextGenerationModel
genModel = TextGenerationModel.from_pretrained("text-bison@001")
result = genModel.predict("Please tell me where is Oklahoma located.")

In [5]:
print(type(result))
print('\n\n%s\n\n' % str(result))
print(type(result.text))
print('\n\n%s' % result.text)

<class 'vertexai.language_models._language_models.TextGenerationResponse'>


Oklahoma is located in the United States. It is bordered by Texas to the south, Arkansas to the east, Missouri to the northeast, Kansas to the north, and Colorado to the northwest. Oklahoma is a state in the South Central region of the United States. It is bordered by Texas to the south, Arkansas to the east, Missouri to the northeast, Kansas to the north, and Colorado to the northwest. Oklahoma is the 20th largest state in the United States, with a total area of 69,898 square miles (181,035 km2). The state's capital


<class 'str'>


Oklahoma is located in the United States. It is bordered by Texas to the south, Arkansas to the east, Missouri to the northeast, Kansas to the north, and Colorado to the northwest. Oklahoma is a state in the South Central region of the United States. It is bordered by Texas to the south, Arkansas to the east, Missouri to the northeast, Kansas to the north, and Colorado to the nor

### Embeddings

In [6]:
embedding_model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

embeddings = embedding_model.get_embeddings(['Hello, this is text'])

In [7]:
print(type(embeddings[0].values))
print('\n\n%s...' % str(embeddings[0].values)[:256])

<class 'list'>


[0.0009353527566418052, 0.013470692560076714, 0.006255208980292082, 0.011925805360078812, 0.01133446954190731, 0.0008504986180923879, 0.0034989851992577314, 0.037757065147161484, 0.0011046730214729905, 0.036481037735939026, -0.0030782767571508884, -0.02927...


### Chat-specific

In [8]:
chat_model = ChatModel.from_pretrained("chat-bison@001")

context = """You speak like a pirate."""

In [9]:
chat = chat_model.start_chat(
    context=context,
    examples=[]
)
parameters = {
    "temperature": 0.2,
    "max_output_tokens": 256,
    "top_p": 0.95,
#     "top_k": 40,
}

In [10]:
chatResponse = chat.send_message('Tell me what is the capital city of Portugal.', **parameters)

In [11]:
print(chatResponse.text)

Ahoy there, matey! The capital city of Portugal is Lisbon. It's a beautiful city with a rich history and culture. If ye ever have the chance to visit, I highly recommend it.
